In [4]:
import fakeredis
import json

server = fakeredis.FakeServer()
redis = fakeredis.FakeStrictRedis(server=server)
print(redis)

userID = 12345
record = redis.get(userID)
print(record)


FakeStrictRedis<ConnectionPool<FakeConnection<server=<fakeredis._server.FakeServer object at 0x7efc807b1860>,db=0>>>
None


In [5]:

# Create 
record = redis.get(userID)
if record is None:
    profile = {"sessions": 0 }
    redis.set(userID, json.dumps(profile))

# Read
record = redis.get(userID)
profile = json.loads(record)
print("Sessions:" + str(profile['sessions']))

# Update
profile['sessions'] += 1
redis.set(userID, json.dumps(profile))

# Expire/Delete 
redis.expire(userID, 15)
redis.delete(userID)


Sessions:0


1

In [6]:
import pandas as pd

df = pd.read_csv("game_skater_stats.csv")
df = df[df['player_id'] == 8467412]
df.head(3)

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2011030221,8467412,1,999,0,0,1,3,0,0,...,0,0,0,0,0,0,-1,885,98,16
36,2011030222,8467412,1,993,0,0,1,4,0,0,...,0,0,0,0,0,1,0,919,54,20
90,2011030223,8467412,1,1091,1,1,4,3,0,0,...,1,0,0,0,0,0,1,1023,68,0


In [7]:
import requests

for index, row in df.iterrows():
    event = { "playerID": int(row['player_id']), "Game_ID": int(row['game_id']),
              "goals": int(row['goals']), "assists": int(row['assists']), 
              "shots": int(row['shots']), "hits": int(row['hits']) }
    print(event)
    
    #requests.post("http://localhost:5000/update", json = event) 
    #requests.get("http://localhost:5000/score?player=8467412")   
    break
    


{'playerID': 8467412, 'Game_ID': 2011030221, 'goals': 0, 'assists': 0, 'shots': 1, 'hits': 3}


In [12]:
import flask
import fakeredis
import json

server = fakeredis.FakeServer()
redis = fakeredis.FakeStrictRedis(server=server)
app = flask.Flask(__name__)

# endpoint for profile updates
@app.route("/update", methods=["GET","POST"])
def update():
    
    # get the player ID to update
    event = flask.request.json
    playerID = event.get('playerID')
    
    # CREATE: heck if a record exists 
    record = redis.get(playerID)
    if record is None:
        profile = {"goals": 0, "shots": 0, "assists": 0, "hits": 0 }
        redis.set(playerID, json.dumps(profile))

    # READ: get the user summary
    record = redis.get(playerID)
    profile = json.loads(record)

    # UPDATE: add the new attributes
    profile['goals'] += event['goals']
    profile['shots'] += event['shots']
    profile['assists'] += event['assists']
    profile['hits'] += event['hits']
    redis.set(playerID, json.dumps(profile))
    
    # return the updated profile
    return flask.jsonify(profile)

# endpoint for model serving
@app.route("/score", methods=["GET"])
def score():
    result = {}

    try:
        # get the user profile 
        playerID = flask.request.args['playerID']
        record = redis.get(playerID)
        profile = json.loads(record)
        
        # calculate a regression value
        score = 1 + profile['goals']   * 10.0 \
                  + profile['shots']   * 1.0 \
                  + profile['assists'] * 2.0 \
                  + profile['hits']    * 0.5
        
        result['score'] = score
    except:
        None
    
    return flask.jsonify(result)

# start the flask app, allow remote connections
if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2020 18:04:24] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:24] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:30] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:30] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:31] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:31] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:31] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:31] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:32] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:32] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:33] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:33] "GET /score?playerID=8467412 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2020 18:04:34] "POST /upda